In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import qutip
from CoupledQuantumSystems.drive import *
from scipy.optimize import minimize
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process
import os, pickle

In [2]:
EJ = 4
EC = EJ/2
EL = EJ/30

qbt = gfIFQ(EJ = EJ,EC = EC, EL = EL, flux = 0,truncated_dim=20)
e_ops = [qutip.basis(qbt.truncated_dim, i)*qutip.basis(qbt.truncated_dim, i).dag() for i in range(10)]

state_0_dressed = qutip.basis(qbt.truncated_dim, 0)
state_1_dressed = qutip.basis(qbt.truncated_dim, 2)
state_plus_X_dressed = (state_0_dressed +  state_1_dressed).unit()
state_minus_X_dressed = (state_0_dressed - state_1_dressed).unit()
state_plus_Y_dressed = (state_0_dressed + 1j * state_1_dressed).unit()
state_minus_Y_dressed = (state_0_dressed - 1j * state_1_dressed).unit()

initial_states  = [
                state_0_dressed,
                state_1_dressed,
                state_plus_X_dressed,
                state_minus_X_dressed,
                state_plus_Y_dressed,
                state_minus_Y_dressed
                ]

e_ops = [qutip.ket2dm(qutip.basis(qbt.truncated_dim, i)) for i in range(qbt.truncated_dim)]

def truncate_to_computational_and_renormaize(state,i,j):
    if state.isket:
        state = qutip.ket2dm(state)
    arr = state.full()
    arr= arr[[i, j], :][:, [i, j]]
    dm = qutip.Qobj(arr)
    dm = dm/dm.tr()
    return dm

def truncate_to_computational_without_renormaize(state,i,j):
    if state.isket:
        state = qutip.ket2dm(state)
    arr = state.full()
    arr= arr[[i, j], :][:, [i, j]]
    dm = qutip.Qobj(arr)
    return dm


zero = qutip.basis(2, 0)
one = qutip.basis(2, 1)
states_ideal  = [zero,
                one,
                (zero +  one).unit(),
                (zero - one).unit(),
                (zero + 1j * one).unit(),
                (zero - 1j *one).unit()]

states_ideal_after_gate = []
for state in states_ideal:
    states_ideal_after_gate.append(qutip.qip.operations.x_gate() * state)

def calc_average_fidelity_with_phase(phase,dms,states_ideal_after_gate):
    gate = qutip.qip.operations.phasegate(phase)
    fid=[]
    for dm,ket_ideal in zip(dms,states_ideal_after_gate):
        fid.append(qutip.fidelity(gate*dm*gate.dag(), ket_ideal))
    return 1-sum(fid)/len(fid)


detuning_arr = np.linspace(0.1,0.4,61)
t_duration_arr = np.linspace(50,125,11)

In [ ]:
EJ = 4
EC = EJ/2
EL = EJ/30

qbt = gfIFQ(EJ=EJ, EC=EC, EL=EL, flux=0, truncated_dim=20)
e_ops = [qutip.ket2dm(qutip.basis(qbt.truncated_dim, i)) for i in range(4)]


initial_states = [qutip.basis(qbt.truncated_dim, 0), qutip.basis(qbt.truncated_dim, 2)]
def objective(detuning, t_duration, amp1_scaling_factor, amp2_scaling_factor):
    detuning1 = detuning
    detuning2 = detuning
    tlist = np.linspace(0,t_duration,100)

    drive_terms = qbt.get_Raman_DRAG_drive_terms(
            i = 0,
            j = 3,
            k = 2,
            detuning1=detuning1,
            detuning2 = detuning2,
            t_duration=t_duration,
            shape='sin^2',
            amp_scaling_factor = 1,
            amp1_scaling_factor = amp1_scaling_factor,
            amp2_scaling_factor = amp2_scaling_factor,
            amp1_correction_scaling_factor = 0,
            amp2_correction_scaling_factor = 0,
        )

    results = []
    for init_state in initial_states:
        res = ODEsolve_and_post_process(
            y0=init_state,
            tlist=tlist,
            static_hamiltonian=qbt.diag_hamiltonian,
            drive_terms=drive_terms,
            e_ops=e_ops,
            print_progress=False,
        )
        results.append(res)

    one_minus_pop2 = np.abs( 1- (results[0].expect[2][-1] + 0.99* results[0].expect[1][-1]))
    one_minus_pop0 = np.abs(1- (results[1].expect[0][-1] +  0.99* results[1].expect[1][-1]))
    return one_minus_pop2 + one_minus_pop0


from scipy.optimize import minimize
def rerun_optimization(detuning_idx, t_duration_idx):
    if detuning_idx == 9 or  detuning_idx == 60:
        old_file  = f'json_files/nevergrad_optimized_gf_raman_{detuning_arr[detuning_idx-1]}_{t_duration_arr[0]}.json'
    else:
        old_file  = f'json_files/nevergrad_optimized_gf_raman_{detuning_arr[detuning_idx]}_{t_duration_arr[t_duration_idx]}.json'
    new_file = f'fine_tunedjson_files/nevergrad_optimized_gf_raman_{detuning_arr[detuning_idx]}_{t_duration_arr[t_duration_idx]}.json'
    def wrapped_objective(x):
        amp1_scaling_factor = x[0]
        amp2_scaling_factor = x[1]
        value = objective(detuning_arr[detuning_idx], t_duration_arr[t_duration_idx], amp1_scaling_factor, amp2_scaling_factor)
        # print(f"amp1_scaling_factor: {amp1_scaling_factor}, amp2_scaling_factor: {amp2_scaling_factor}, value: {value}")
        return value

    old_result = json.load(open(old_file))
    result = minimize(wrapped_objective, x0=[old_result['best_amp1'], old_result['best_amp2']], method='L-BFGS-B')
    new_result = {
        'detuning': detuning_arr[detuning_idx],
        't_duration': t_duration_arr[t_duration_idx],
        'best_amp1': result.x[0],
        'best_amp2': result.x[1],
        'best_value': result.fun,}
    with open(new_file, 'w') as f:
        json.dump(new_result, f)
    return result

from tqdm.notebook import tqdm
for detuning_idx in tqdm(range(45,len(detuning_arr)),desc='Rerunning optimizations'):
    t_duration_idx = 0
    rerun_optimization(detuning_idx, t_duration_idx)


Rerunning optimizations:   0%|          | 0/16 [00:00<?, ?it/s]